In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import regex
import datetime
import matplotlib.pyplot as plt

import warnings
warnings.simplefilter(action='ignore')

# Show 2 decimals
pd.set_option('display.float_format', lambda x: '%.0f' % x)
#pd.options.display.float_format = '{:,.2f}'.format

#Butun kolon ve satirlari gormek icin
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# Set plot background
sns.set_theme(style="whitegrid")

# Euro sign €
#print ("%s"%(u"\N{euro sign}"))

In [2]:
auto = pd.read_csv("/Users/macbook/Desktop/csv_dosyalari/autoscout_data_2000.csv")
df = auto.copy()
df.head(2)

,location,url,make,model,short_description,price,mileage,gearbox,registration,fuel_type,power,seller,body_type,type,drivetrain,seats,doors,country_version,offer_number,warranty,Mileage,first_registration,production_date,general_inspection,last_service,full_service_history,non_smoker_vehicle,previous_owner,last_timing_belt_change,Power,Gearbox,engine_size,gears,cylinders,empty_weight,Fuel_type,fuel_consumption,co2_emissions,emission_class,emissions_sticker,\nComfort & Convenience\n,\nEntertainment & Media\n,\nSafety & Security\n,extras,\ncolour,\nmanufacturer_color,\npaint,upholstery_colour,upholstery
0,"LOON OP ZAND, NL",https://www.autoscout24.com/offers/audi-a3-2-0...,"\n,Audi ,\n","\n,A3,\n",2.0 FSI Ambition Pro Line,"[['\n', '€ 4,450.-', '\n']]","['', ['191,066 km']]","\n,Automatic,\n",08/2003,"\n,Gasoline,\n","110 kW (150 hp),","['\n', ['Dealer\n']]","\n,Compact,\n","\n,Used,\n","\n,Front,\n",\n5,\n3,"Country version,None",\nSJ-112-R,"\n,6 months,\n","[['\n'], ['']]",08/2003,None,05/2022,None,"\n,Yes,\n","\n,None,\n","[['\n'], ['None']]","[['\n'], ['None']]","\n,110 kW (150 hp),\n","\n,Automatic,\n","\n,1,984 cc,\n","\n,None,\n","\n,4,\n","\n,1,280 kg,\n","\n,Super 95,\n","\n,None,\n","['\n', [' ', ' ', ' '], '\n']","\n,Euro 4,\n","\n,None,\n","[['Air conditioning', 'Armrest', 'Automatic cl...","[['CD player', 'Radio']]","[['ABS', 'Alarm system', 'Central door lock', ...",[['Alloy wheels']],"\n4, Grey",None,"\npaint, Metallic","\nupholstery, Black","\n8, Cloth"
1,"HEERLEN, NL",https://www.autoscout24.com/offers/audi-a3-spo...,"\n,Audi ,\n","\n,A3,\n",Sportback 1.4 TFSI Ambiente/ECC/PDC//Lichtmeta...,"[['\n', '€ 12,950.-', '\n']]","['', ['94,758 km']]","\n,Automatic,\n",10/2013,"\n,Gasoline,\n","92 kW (125 hp),","['\n', ['Dealer\n']]","\n,Compact,\n","\n,Used,\n","\n,Front,\n",\n5,\n5,"Country version,None",\n809725,"\n,None,\n","[['\n'], ['']]",10/2013,None,None,None,"\n,Yes,\n","\n,None,\n","[['\n'], ['None']]","[['\n'], ['None']]","\n,92 kW (125 hp),\n","\n,Automatic,\n","\n,1,395 cc,\n","\n,None,\n","\n,4,\n","\n,1,200 kg,\n","\n,Gasoline,\n","['\n', ['4.9 l/100 km (comb.)', ' ', ' '], '\n']","\n,None,\n","\n,None,\n","\n,None,\n","[['Air conditioning', 'Automatic climate contr...","[['CD player', 'Radio']]","[['Alarm system', 'Central door lock', 'Centra...","[['Alloy wheels (16"")']]","\n4, White",None,None,"\nupholstery, Black","\n8, Cloth"


In [3]:
# Drop Duplicated Columns and Url column
df.drop(['url', 'Mileage', 'Power', 'gearbox'], axis=1, inplace= True)

df.columns

Index(['location', 'make', 'model', 'short_description', 'price', 'mileage',
       'registration', 'fuel_type', 'power', 'seller', 'body_type', 'type',
       'drivetrain', 'seats', 'doors', 'country_version', 'offer_number',
       'warranty', 'first_registration', 'production_date',
       'general_inspection', 'last_service', 'full_service_history',
       'non_smoker_vehicle', 'previous_owner', 'last_timing_belt_change',
       'Gearbox', 'engine_size', 'gears', 'cylinders', 'empty_weight',
       'Fuel_type', 'fuel_consumption', 'co2_emissions', 'emission_class',
       'emissions_sticker', '\nComfort & Convenience\n',
       '\nEntertainment & Media\n', '\nSafety & Security\n', 'extras',
       '\ncolour', '\nmanufacturer_color', '\npaint', ' upholstery_colour ',
       ' upholstery '],
      dtype='object')

# Cleansing 

In [4]:
# 1 - Rename and Reassign Columns (move unwanted characters from column names)
df.columns = ['location', 'make', 'model', 'short_description', 'price','mileage_km', 'registration', 'fuel_type', 'power', 'seller',
       'body_type', 'type', 'drivetrain', 'seats', 'doors', 'country_version','offer_number', 'warranty_months',  'first_registration',
       'production_date', 'general_inspection', 'last_service','full_service_history', 'non_smoker_vehicle', 'previous_owner',
       'last_timing_belt_change', 'Gearbox', 'engine_size_cc', 'gears','cylinders', 'empty_weight_kg', 'Fuel_type', 'fuel_consumption',
       'co2_emissions_g_km_comb', 'emission_class', 'emissions_sticker','Comfort & Convenience', 'Entertainment & Media',
       'Safety & Security', 'extras', 'colour', 'manufacturer_color','paint', 'upholstery_colour', 'upholstery']

In [5]:
# 2 - Removing unwanted Chracters from data
# Except these colums ====> location, short_description, registration, first_registration, general_inspection, Comfort & Convenience, Entertainment & Media, Safety & Security, extras
clmns = ['make', 'model', 'price','mileage_km', 'fuel_type', 'power', 'seller',
       'body_type', 'type', 'drivetrain', 'seats', 'doors', 'country_version','offer_number', 'warranty_months',
       'production_date', 'last_service','full_service_history', 'non_smoker_vehicle', 'previous_owner',
       'last_timing_belt_change', 'Gearbox', 'engine_size_cc', 'gears','cylinders', 'empty_weight_kg', 'Fuel_type', 'fuel_consumption',
       'co2_emissions_g_km_comb', 'emission_class', 'emissions_sticker',
       'colour', 'manufacturer_color','paint', 'upholstery_colour', 'upholstery']

for column in clmns:
    df[column] = df[column].str.replace(r'\W', '')
    
df.head(2)

,location,make,model,short_description,price,mileage_km,registration,fuel_type,power,seller,body_type,type,drivetrain,seats,doors,country_version,offer_number,warranty_months,first_registration,production_date,general_inspection,last_service,full_service_history,non_smoker_vehicle,previous_owner,last_timing_belt_change,Gearbox,engine_size_cc,gears,cylinders,empty_weight_kg,Fuel_type,fuel_consumption,co2_emissions_g_km_comb,emission_class,emissions_sticker,Comfort & Convenience,Entertainment & Media,Safety & Security,extras,colour,manufacturer_color,paint,upholstery_colour,upholstery
0,"LOON OP ZAND, NL",Audi,A3,2.0 FSI Ambition Pro Line,n4450n,191066km,08/2003,Gasoline,110kW150hp,nDealern,Compact,Used,Front,5,3,CountryversionNone,SJ112R,6months,08/2003,None,05/2022,None,Yes,None,nNone,nNone,Automatic,1984cc,None,4,1280kg,Super95,None,nn,Euro4,None,"[['Air conditioning', 'Armrest', 'Automatic cl...","[['CD player', 'Radio']]","[['ABS', 'Alarm system', 'Central door lock', ...",[['Alloy wheels']],4Grey,None,paintMetallic,upholsteryBlack,8Cloth
1,"HEERLEN, NL",Audi,A3,Sportback 1.4 TFSI Ambiente/ECC/PDC//Lichtmeta...,n12950n,94758km,10/2013,Gasoline,92kW125hp,nDealern,Compact,Used,Front,5,5,CountryversionNone,809725,None,10/2013,None,None,None,Yes,None,nNone,nNone,Automatic,1395cc,None,4,1200kg,Gasoline,n49l100kmcombn,None,None,None,"[['Air conditioning', 'Automatic climate contr...","[['CD player', 'Radio']]","[['Alarm system', 'Central door lock', 'Centra...","[['Alloy wheels (16"")']]",4White,None,None,upholsteryBlack,8Cloth


In [6]:
# 3 - Delete NL from Location
df.location = df['location'].str.replace(', NL', '').str.upper()
df.location = df['location'].str.replace('.', '').str.replace('5', '').str.replace('6', '').str.replace(r'\+318429897', '')
df.location = df.location.replace('', np.NaN)

# Delete n from columns
L01 = ['seller', 'price', 'previous_owner', 'last_timing_belt_change', 'fuel_consumption', 'co2_emissions_g_km_comb']
for i in L01:
    df[i] = df[i].str.replace('n', '')

# Model nNonen to np.NaN
df.model = df.model.str.replace('nNonen', '').replace('', np.NaN)

# Delete [ ] from L02    
L02 = ['Comfort & Convenience', 'Entertainment & Media', 'Safety & Security', 'extras']
for i in L02:
    df[i] = df[i].str.replace('[', '')
    df[i] = df[i].str.replace(']', '')
    
# delete 4 from colour
df['colour'] = df['colour'].str.replace('4', '')

# Delete paint from paint column
df['paint'] = df['paint'].str.replace('paint', '')

# Delete upholstery from upholstery_colour
df['upholstery_colour'] = df['upholstery_colour'].str.replace('upholstery', '')

# Delete 8 from upholstery
df['upholstery'] = df['upholstery'].str.replace('8', '')

df.head(2)

,location,make,model,short_description,price,mileage_km,registration,fuel_type,power,seller,body_type,type,drivetrain,seats,doors,country_version,offer_number,warranty_months,first_registration,production_date,general_inspection,last_service,full_service_history,non_smoker_vehicle,previous_owner,last_timing_belt_change,Gearbox,engine_size_cc,gears,cylinders,empty_weight_kg,Fuel_type,fuel_consumption,co2_emissions_g_km_comb,emission_class,emissions_sticker,Comfort & Convenience,Entertainment & Media,Safety & Security,extras,colour,manufacturer_color,paint,upholstery_colour,upholstery
0,LOON OP ZAND,Audi,A3,2.0 FSI Ambition Pro Line,4450,191066km,08/2003,Gasoline,110kW150hp,Dealer,Compact,Used,Front,5,3,CountryversionNone,SJ112R,6months,08/2003,None,05/2022,None,Yes,None,Noe,Noe,Automatic,1984cc,None,4,1280kg,Super95,Noe,,Euro4,None,"'Air conditioning', 'Armrest', 'Automatic clim...","'CD player', 'Radio'","'ABS', 'Alarm system', 'Central door lock', 'C...",'Alloy wheels',Grey,None,Metallic,Black,Cloth
1,HEERLEN,Audi,A3,Sportback 1.4 TFSI Ambiente/ECC/PDC//Lichtmeta...,12950,94758km,10/2013,Gasoline,92kW125hp,Dealer,Compact,Used,Front,5,5,CountryversionNone,809725,None,10/2013,None,None,None,Yes,None,Noe,Noe,Automatic,1395cc,None,4,1200kg,Gasoline,49l100kmcomb,Noe,None,None,"'Air conditioning', 'Automatic climate control...","'CD player', 'Radio'","'Alarm system', 'Central door lock', 'Central ...","'Alloy wheels (16"")'",White,None,None,Black,Cloth


In [7]:
df['mileage_km'] = df['mileage_km'].str.replace('km', '')
df['warranty_months'] = df['warranty_months'].str.replace('months', '')
df['engine_size_cc'] = df['engine_size_cc'].str.replace('cc', '')
df['empty_weight_kg'] = df['empty_weight_kg'].str.replace('kg', '')
df['country_version'] = df['country_version'].str.replace('Countryversion', '')
df['co2_emissions_g_km_comb'] = df['co2_emissions_g_km_comb'].str.replace('gkmcomb', '')
df.head(1)

,location,make,model,short_description,price,mileage_km,registration,fuel_type,power,seller,body_type,type,drivetrain,seats,doors,country_version,offer_number,warranty_months,first_registration,production_date,general_inspection,last_service,full_service_history,non_smoker_vehicle,previous_owner,last_timing_belt_change,Gearbox,engine_size_cc,gears,cylinders,empty_weight_kg,Fuel_type,fuel_consumption,co2_emissions_g_km_comb,emission_class,emissions_sticker,Comfort & Convenience,Entertainment & Media,Safety & Security,extras,colour,manufacturer_color,paint,upholstery_colour,upholstery
0,LOON OP ZAND,Audi,A3,2.0 FSI Ambition Pro Line,4450,191066,08/2003,Gasoline,110kW150hp,Dealer,Compact,Used,Front,5,3,None,SJ112R,6,08/2003,None,05/2022,None,Yes,None,Noe,Noe,Automatic,1984,None,4,1280,Super95,Noe,,Euro4,None,"'Air conditioning', 'Armrest', 'Automatic clim...","'CD player', 'Radio'","'ABS', 'Alarm system', 'Central door lock', 'C...",'Alloy wheels',Grey,None,Metallic,Black,Cloth


In [8]:
df1 = df.copy()

L03 = ['price', 'mileage_km','engine_size_cc', 'empty_weight_kg', 'seats', 'doors', 'warranty_months', 'cylinders', 'co2_emissions_g_km_comb']
for i in L03:
    df[i] = df[i].replace(['None', 'Noe', 'Yes', 'Automatic', 'Manual', 'Semiautomatic'], np.NaN)
    df[i] = df[i].replace('', np.NaN)
    df[i] = df[i].astype(float)

df.describe().T

,count,mean,std,min,25%,50%,75%,max
price,69534,24369,468620,1,8945,17450,28935,123456789
mileage_km,70951,107148,88475,1,30194,95615,163794,2215714
seats,65562,5,1,1,5,5,5,14
doors,69790,5,1,1,4,5,5,6
warranty_months,24004,8,9,1,6,6,6,84
engine_size_cc,65265,1771,702,0,1390,1598,1995,65535
cylinders,64809,4,1,1,4,4,4,12
empty_weight_kg,69500,1433,321,1,1234,1382,1591,4200
co2_emissions_g_km_comb,52190,129,68,0,105,133,164,5900


In [9]:
L04 = ['power_kw', 'power_hp']
df[L04] = df.power.str.split('kW', expand= True)   
df['power_hp'] = df['power_hp'].str.replace('hp', '').replace('None', np.NaN) 
df['power_kw'] = df['power_kw'].str.replace('kg', '').replace('None', np.NaN)     
df[L04] = df[L04].astype(float)
df = df.drop(['power'], axis=1)
df.head(1)

,location,make,model,short_description,price,mileage_km,registration,fuel_type,seller,body_type,type,drivetrain,seats,doors,country_version,offer_number,warranty_months,first_registration,production_date,general_inspection,last_service,full_service_history,non_smoker_vehicle,previous_owner,last_timing_belt_change,Gearbox,engine_size_cc,gears,cylinders,empty_weight_kg,Fuel_type,fuel_consumption,co2_emissions_g_km_comb,emission_class,emissions_sticker,Comfort & Convenience,Entertainment & Media,Safety & Security,extras,colour,manufacturer_color,paint,upholstery_colour,upholstery,power_kw,power_hp
0,LOON OP ZAND,Audi,A3,2.0 FSI Ambition Pro Line,4450,191066,08/2003,Gasoline,Dealer,Compact,Used,Front,5,3,None,SJ112R,6,08/2003,None,05/2022,None,Yes,None,Noe,Noe,Automatic,1984,None,4,1280,Super95,Noe,NaN,Euro4,None,"'Air conditioning', 'Armrest', 'Automatic clim...","'CD player', 'Radio'","'ABS', 'Alarm system', 'Central door lock', 'C...",'Alloy wheels',Grey,None,Metallic,Black,Cloth,110,150


# Dulpicated

In [10]:
df.shape

(71104, 46)

In [11]:
df.duplicated().sum()

2012

In [12]:
df.loc[df.duplicated(), :].head(2)

,location,make,model,short_description,price,mileage_km,registration,fuel_type,seller,body_type,type,drivetrain,seats,doors,country_version,offer_number,warranty_months,first_registration,production_date,general_inspection,last_service,full_service_history,non_smoker_vehicle,previous_owner,last_timing_belt_change,Gearbox,engine_size_cc,gears,cylinders,empty_weight_kg,Fuel_type,fuel_consumption,co2_emissions_g_km_comb,emission_class,emissions_sticker,Comfort & Convenience,Entertainment & Media,Safety & Security,extras,colour,manufacturer_color,paint,upholstery_colour,upholstery,power_kw,power_hp
638,WOMMELS,Audi,A3,1.6 FSI Ambiente,2750,289500,11/2006,Gasoline,Privateseller,Compact,Used,None,5,4,None,None,NaN,11/2006,None,06/2022,None,Yes,None,Noe,Noe,Manual,1598,6,4,1240,Gasoline,67l100kmcomb89l100kmcity54l100kmcoutry,161,Euro4,None,"'Air conditioning', 'Automatic climate control...","'Hands-free equipment', 'On-board computer', '...","'Central door lock', 'Central door lock with r...","'Alloy wheels (16"")', 'Automatically dimming i...",Blue,None,None,None,None,85,116
1199,NAARDEN,Audi,TTS,Roadster 2.0 TFSI quattro Hoofdruimteverwarmin...,49900,5076,10/2017,Gasoline,Dealer,Convertible,Used,4WD,2,2,None,PV399Z,6,10/2017,None,New,None,Yes,None,Noe,Noe,Automatic,1984,6,4,1445,Super95,69l100kmcomb84l100kmcity6l100kmcoutry,159,Euro6,None,"'Air conditioning', 'Cruise control', 'Electri...","'Digital cockpit', 'Hands-free equipment', 'Ra...","'ABS', 'Alarm system', 'Bi-Xenon headlights', ...","'Alloy wheels (18"")', 'Automatically dimming i...",Black,None,Metallic,Black,Fullleather,228,310


In [13]:
print(df.shape)
df.drop_duplicates(inplace= True)

(71104, 46)


In [14]:
df.shape

(69092, 46)

In [15]:
df.duplicated().sum()

0

### Price Null DataFrame

In [16]:
# Find the rows which have no price

price_null_tf = (df['price'].isnull())
price_null = (df[price_null_tf])
price_null.shape

(1523, 46)

### Drop Empty Price, Make , Year Columns

In [17]:
df.shape

(69092, 46)

In [18]:
#df[['price', 'make', 

In [19]:
df.dropna(subset=['price'], inplace= True)
df.shape

(67569, 46)

## DF_Dates 

In [20]:
L05 = ['registration', 'first_registration', 'general_inspection']
for i in L05:
    df[i] = pd.to_datetime(df[i], format= '%m/%Y', errors= 'coerce')

df['production_date'] = pd.to_datetime(df['production_date'], format= '%Y', errors= 'coerce')

# Creating new Features df_dates
df_dates = pd.DataFrame({'registration':  df.registration,
                         'first_registration' : df.first_registration,
                         'production_date' : df.production_date,
                         'general_inspection' : df.general_inspection,
                        'last_timing_belt_change' : df.last_timing_belt_change})

L05 = ['registration', 'first_registration', 'general_inspection']
for i in L05:
    df_dates[i] = pd.to_datetime(df_dates[i], format= '%m/%Y', errors= 'coerce')

df_dates['production_date'] = pd.to_datetime(df_dates['production_date'], format= '%Y', errors= 'coerce')
df_dates['last_timing_belt_change'] = pd.to_datetime(df_dates['last_timing_belt_change'], format= '%m%Y', errors= 'coerce')
df_dates.head()

,registration,first_registration,production_date,general_inspection,last_timing_belt_change
0,2003-08-01,2003-08-01,NaT,2022-05-01,NaT
1,2013-10-01,2013-10-01,NaT,NaT,NaT
2,2013-05-01,2013-05-01,NaT,2022-03-01,NaT
3,2015-06-01,2015-06-01,2015-01-01,NaT,NaT
4,2015-10-01,2015-10-01,NaT,NaT,NaT


In [21]:
df2 = df.copy()

In [22]:
# Add year Column !!!!!!
pd.options.display.float_format = '{:.0f}'.format 
df.insert(4, 'year', df.registration.dt.year, True )
df.insert(3, 'make_model', (df['make'].astype(str) +"-"+ df["model"].astype(str)), True)
df.head(2)

,location,make,model,make_model,short_description,year,price,mileage_km,registration,fuel_type,seller,body_type,type,drivetrain,seats,doors,country_version,offer_number,warranty_months,first_registration,production_date,general_inspection,last_service,full_service_history,non_smoker_vehicle,previous_owner,last_timing_belt_change,Gearbox,engine_size_cc,gears,cylinders,empty_weight_kg,Fuel_type,fuel_consumption,co2_emissions_g_km_comb,emission_class,emissions_sticker,Comfort & Convenience,Entertainment & Media,Safety & Security,extras,colour,manufacturer_color,paint,upholstery_colour,upholstery,power_kw,power_hp
0,LOON OP ZAND,Audi,A3,Audi-A3,2.0 FSI Ambition Pro Line,2003,4450,191066,2003-08-01,Gasoline,Dealer,Compact,Used,Front,5,3,None,SJ112R,6,2003-08-01,NaT,2022-05-01,None,Yes,None,Noe,Noe,Automatic,1984,None,4,1280,Super95,Noe,NaN,Euro4,None,"'Air conditioning', 'Armrest', 'Automatic clim...","'CD player', 'Radio'","'ABS', 'Alarm system', 'Central door lock', 'C...",'Alloy wheels',Grey,None,Metallic,Black,Cloth,110,150
1,HEERLEN,Audi,A3,Audi-A3,Sportback 1.4 TFSI Ambiente/ECC/PDC//Lichtmeta...,2013,12950,94758,2013-10-01,Gasoline,Dealer,Compact,Used,Front,5,5,None,809725,NaN,2013-10-01,NaT,NaT,None,Yes,None,Noe,Noe,Automatic,1395,None,4,1200,Gasoline,49l100kmcomb,NaN,None,None,"'Air conditioning', 'Automatic climate control...","'CD player', 'Radio'","'Alarm system', 'Central door lock', 'Central ...","'Alloy wheels (16"")'",White,None,None,Black,Cloth,92,125


### Combine Make Column to Make_Counrty Column

In [23]:
german = ['Audi', 'MercedesBenz', 'BMW', 'Volkswagen', 'Opel', 'Skoda']
japan = ['Toyota', 'Honda', 'Mazda']
sweden = ['Volvo']
france = ['Renault', 'Peugeot', 'Citroen']
america = ['Ford', 'Chevrolet', 'Tesla']
south_korea = ['Kia', 'Hyundai']
italy = ['Fiat']
romanian = ['Dacia']

conditions = [ 
    (df.make.isin(german)),
    (df.make.isin(japan)),
    (df.make.isin(sweden)),
    (df.make.isin(france)),
    (df.make.isin(america)),
    (df.make.isin(south_korea)),
    (df.make.isin(italy)),
    (df.make.isin(romanian)),
]

values = ['German', 'Japan', 'Sweden', 'France', 'America', 'South_Korea', 'Italy', 'Romanian']

df['make_country'] = np.select(conditions, values)
df.tail(1)

,location,make,model,make_model,short_description,year,price,mileage_km,registration,fuel_type,seller,body_type,type,drivetrain,seats,doors,country_version,offer_number,warranty_months,first_registration,production_date,general_inspection,last_service,full_service_history,non_smoker_vehicle,previous_owner,last_timing_belt_change,Gearbox,engine_size_cc,gears,cylinders,empty_weight_kg,Fuel_type,fuel_consumption,co2_emissions_g_km_comb,emission_class,emissions_sticker,Comfort & Convenience,Entertainment & Media,Safety & Security,extras,colour,manufacturer_color,paint,upholstery_colour,upholstery,power_kw,power_hp,make_country
71103,SOESTERBERG,Volvo,S60,Volvo-S60,"2.4 Airco, Cruise Control, Stuurbekrachtiging",2001,944,404758,2001-09-01,LPG,Dealer,Sedan,Used,Front,5,4,None,26HNLS,NaN,2001-09-01,NaT,NaT,None,None,None,Noe,Noe,Manual,2435,5,5,1488,LiquidpetroleumgasLPG,84l100kmcomb116l100kmcity65l100kmcoutry,207,Euro3,None,"'Air conditioning', 'Armrest', 'Automatic clim...","'On-board computer', 'Radio'","'ABS', 'Alarm system', 'Central door lock', 'C...",'Alloy wheels',Grey,None,Metallic,None,Partleather,104,141,Sweden


In [24]:
df.price.isnull().sum()

0

# Wrong Values to NaN

In [25]:
# Year 2025 and 2028 to np.nan
df['year'] = df['year'].replace([2025, 2028], np.NaN)
df.insert(4, 'age', (2022 - df.year), True)
df.head(2)

,location,make,model,make_model,age,short_description,year,price,mileage_km,registration,fuel_type,seller,body_type,type,drivetrain,seats,doors,country_version,offer_number,warranty_months,first_registration,production_date,general_inspection,last_service,full_service_history,non_smoker_vehicle,previous_owner,last_timing_belt_change,Gearbox,engine_size_cc,gears,cylinders,empty_weight_kg,Fuel_type,fuel_consumption,co2_emissions_g_km_comb,emission_class,emissions_sticker,Comfort & Convenience,Entertainment & Media,Safety & Security,extras,colour,manufacturer_color,paint,upholstery_colour,upholstery,power_kw,power_hp,make_country
0,LOON OP ZAND,Audi,A3,Audi-A3,19,2.0 FSI Ambition Pro Line,2003,4450,191066,2003-08-01,Gasoline,Dealer,Compact,Used,Front,5,3,None,SJ112R,6,2003-08-01,NaT,2022-05-01,None,Yes,None,Noe,Noe,Automatic,1984,None,4,1280,Super95,Noe,NaN,Euro4,None,"'Air conditioning', 'Armrest', 'Automatic clim...","'CD player', 'Radio'","'ABS', 'Alarm system', 'Central door lock', 'C...",'Alloy wheels',Grey,None,Metallic,Black,Cloth,110,150,German
1,HEERLEN,Audi,A3,Audi-A3,9,Sportback 1.4 TFSI Ambiente/ECC/PDC//Lichtmeta...,2013,12950,94758,2013-10-01,Gasoline,Dealer,Compact,Used,Front,5,5,None,809725,NaN,2013-10-01,NaT,NaT,None,Yes,None,Noe,Noe,Automatic,1395,None,4,1200,Gasoline,49l100kmcomb,NaN,None,None,"'Air conditioning', 'Automatic climate control...","'CD player', 'Radio'","'Alarm system', 'Central door lock', 'Central ...","'Alloy wheels (16"")'",White,None,None,Black,Cloth,92,125,German


# PRICE (Last One)

In [26]:
# Wrong Price to NaN

# high price to NaN
#index_list = [1406, 22706, 23072, 66537, 71027]
#df.loc[index_list, 'price'] = np.NaN

# low price to NaN


print(df.price.max())
df.price.isnull().sum()

123456789.0


0

In [27]:
list_short = ['make', 'model', 'price', 'year','age','mileage_km']
df.loc[(df.price <=500) & (df.mileage_km <=100000) , list_short]

,make,model,price,year,age,mileage_km
14588,Ford,KaKa,499,2001,21,31949
17729,Ford,Transit,275,2017,5,77521
50269,Kia,Picanto,239,2022,0,10
52116,Kia,eNiro,464,2022,0,10
52122,Kia,eNiro,484,2022,0,10


In [28]:
df.loc[(df.price <=5000) & (df.mileage_km <=10000) , list_short]

,make,model,price,year,age,mileage_km
9896,BMW,116,4500,2006,16,186
23050,MercedesBenz,Vito,4700,2007,15,250
23882,MercedesBenz,300,2700,2000,22,3000
32155,Volkswagen,Golf,4900,2010,12,1
32160,Volkswagen,Polo,4900,2010,12,1
50269,Kia,Picanto,239,2022,0,10
52113,Kia,EV6,579,2021,1,10
52116,Kia,eNiro,464,2022,0,10
52118,Kia,eNiro,539,2022,0,10
52122,Kia,eNiro,484,2022,0,10


# Missing Value Make

### Make, Model, Mileage_Km

In [107]:
df.loc[(df.model.isnull()) & (df.year.isnull()) & (df.mileage_km.isnull())].shape

(0, 50)

In [109]:
df.loc[(df.model.isnull()) & (df.year.isnull())].shape

(8, 50)

In [110]:
df.loc[(df.model.isnull()) & (df.mileage_km.isnull())].shape

(0, 50)

In [111]:
df.loc[(df.year.isnull()) & (df.mileage_km.isnull())].shape

(5, 50)

In [112]:
df[['model', 'year', 'mileage_km']].isnull().sum()

model           8
year           48
mileage_km    150
dtype: int64

In [120]:
print(df.shape)
df.dropna(subset=['model', 'year','mileage_km'], inplace= True)
df.shape  # 193 index Deleted

(67569, 50)


(67376, 50)

# Save

In [122]:
df.to_csv('auto_02.csv')

# DF_Mercedes_Bmw

In [145]:
L01 = ['MercedesBenz', 'BMW']
df_mer_bmw = df.loc[df.make.isin(L01), :]
df_mer_bmw.to_csv('df_mer_bmw.csv')
df_mer_bmw.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13956 entries, 6394 to 25486
Data columns (total 50 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   location                 13955 non-null  object        
 1   make                     13956 non-null  object        
 2   model                    13956 non-null  object        
 3   make_model               13956 non-null  object        
 4   age                      13956 non-null  float64       
 5   short_description        13872 non-null  object        
 6   year                     13956 non-null  float64       
 7   price                    13956 non-null  float64       
 8   mileage_km               13956 non-null  float64       
 9   registration             13956 non-null  datetime64[ns]
 10  fuel_type                13956 non-null  object        
 11  seller                   13956 non-null  object        
 12  body_type                1395

In [161]:
df_mer_bmw.make_model.value_counts().sort_values().head()

MercedesBenz-S300     1
MercedesBenz-220      1
MercedesBenz-E420     1
MercedesBenz-S250     1
MercedesBenz-CE220    1
Name: make_model, dtype: int64

# ML (MERCEDES_BMW)

In [ ]:
# Drop the Values smaller than 30 

In [269]:
df_mer_bmw = pd.read_csv('df_mer_bmw.csv')
df_mer_bmw.drop('Unnamed: 0', axis=1, inplace=True)

In [271]:
small_30_tf = (df_mer_bmw.make_model.value_counts() < 30)
small_30_tf[small_30_tf][:5]

BMW-645              True
BMW-430              True
BMW-X4               True
MercedesBenz-A220    True
BMW-135              True
Name: make_model, dtype: bool

In [272]:
list(small_30_tf[:5].index)

['BMW-320', 'BMW-X5', 'MercedesBenz-C180', 'BMW-218', 'BMW-330']

In [273]:
index_criteria = small_30_tf[small_30_tf].index[:10]

In [274]:
small_30_tf[small_30_tf].shape

(203,)

In [275]:
filt = df['make_model'].isin(list(small_30_tf[small_30_tf].index))
df.loc[filt].index

Int64Index([], dtype='int64')

In [276]:
# All steps in one code line
small_30_tf = (df_mer_bmw.make_model.value_counts() < 30)

index_criteria = small_30_tf[small_30_tf].index

list_criteria = list(index_criteria)

criteria = df_mer_bmw['make_model'].map(lambda x :  x in list_criteria) # criteria = df.make_model.isin(list_criteria)

df_mer_bmw[criteria].index

Int64Index([    5,     6,    16,   115,   147,   185,   191,   193,   202,
              204,
            ...
            13718, 13720, 13770, 13775, 13788, 13797, 13923, 13936, 13945,
            13952],
           dtype='int64', length=1561)

In [277]:
df_mer_bmw.shape

(13956, 50)

In [278]:
# Drop less than 30 Values in DF

df_mer_bmw.drop(index= (df_mer_bmw[criteria].index), inplace= True)
df_mer_bmw.shape

(12395, 50)

In [279]:
df = df_mer_bmw.copy()

### Drop The Values Model == Others

In [280]:
df.loc[df.model == 'Others', :].index
df.drop(index= (df.loc[df.model == 'Others', :].index), inplace= True)
df.shape

(12135, 50)

### Make Co2 Column np.NaN while the Fueltype = L01

In [292]:
L01= ['Gasoline', 'Diesel', 'ElectricGasoline', 'ElectricDiesel', 'LPG', 'CNG', 'Others']
index_01 = df.loc[(df.fuel_type.isin(L01)) & (df.co2_emissions_g_km_comb == 0), ['make_model', 'fuel_type', 'co2_emissions_g_km_comb']].index
index_01

Int64Index([   13,    18,   146,   156,   172,   330,   491,   551,   557,
              564,
            ...
            13903, 13904, 13905, 13906, 13907, 13909, 13911, 13915, 13918,
            13922],
           dtype='int64', length=1086)

In [296]:
df.co2_emissions_g_km_comb.loc[index_01].replace(0, np.NaN)[:5]

13    NaN
18    NaN
146   NaN
156   NaN
172   NaN
Name: co2_emissions_g_km_comb, dtype: float64

In [350]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
age,12135,8,6,0,4,7,12,25
year,12135,2014,6,1997,2010,2015,2018,2022
price,12135,27472,23640,600,11250,20485,34900,179659
mileage_km,12135,120935,90720,1,47010,112017,178251,827190
seats,11405,5,1,2,5,5,5,9
doors,11900,4,1,2,4,5,5,6
warranty_months,3204,9,8,1,6,6,6,24
engine_size_cc,11266,2049,585,0,1595,1995,2143,5462
cylinders,11168,4,1,1,4,4,4,8
empty_weight_kg,11801,1572,308,112,1335,1480,1720,3960


### np.NaN wich engine sixe > 6000 

In [328]:
df.engine_size_cc.value_counts().sort_index(ascending= False).head()

65535    1
8592     3
5462     1
5461     2
5000     1
Name: engine_size_cc, dtype: int64

In [333]:
L01 = df.loc[df.engine_size_cc > 6000].index
L01

Int64Index([11711, 11893, 12012, 12708], dtype='int64')

In [349]:
df.engine_size_cc.loc[L01] = np.NaN

df.engine_size_cc.loc[L01]

11711   NaN
11893   NaN
12012   NaN
12708   NaN
Name: engine_size_cc, dtype: float64

### Mileage Control

In [368]:
df.loc[(df.mileage_km < 2500) & (df.year < 2020) , ['make_model', 'year', 'mileage_km']]

,make_model,year,mileage_km
792,BMW-640,2011,1
3343,BMW-116,2006,186
3493,BMW-M3,2017,345
4335,BMW-116,2014,1
4732,BMW-X3,2012,1
5086,BMW-525,2005,328
5582,BMW-525,2010,1
5585,BMW-520,2013,1
9517,MercedesBenz-C180,2008,1
11608,MercedesBenz-Vito,2007,250
